In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
p_gwl = Path("../data/AquiMod_simobs_Gretna.csv")
p_met = Path("../data/ukcp18_simobs_Gretna.csv")
df_gwl = pd.read_csv(p_gwl, parse_dates=["Date"], dayfirst=True)
df_met = pd.read_csv(p_met, parse_dates=["Date"], dayfirst=True)
df_data = pd.merge(left=df_met, right=df_gwl, on=["Borehole", "Model", "Date"], how="inner").dropna()
df_data

,Borehole,Model,Date,precipwsnow,PET,Sim,Obs
11419,Gretna,AquiMod,1993-04-07,0.096710,1.530000,39.9447,40.084
11420,Gretna,AquiMod,1993-04-08,22.228661,1.530000,39.9812,40.082
11421,Gretna,AquiMod,1993-04-09,9.274128,1.530000,40.0087,40.106
11422,Gretna,AquiMod,1993-04-10,0.089421,1.530000,40.0106,40.121
11423,Gretna,AquiMod,1993-04-11,1.071286,1.530000,40.0064,40.135
...,...,...,...,...,...,...,...
20814,Gretna,AquiMod,2018-12-27,0.153541,0.248387,39.9721,39.959
20815,Gretna,AquiMod,2018-12-28,1.296672,0.248387,39.9695,39.953
20816,Gretna,AquiMod,2018-12-29,1.978836,0.248387,39.9697,39.950
20817,Gretna,AquiMod,2018-12-30,0.029849,0.248387,39.9671,39.941


In [4]:
fig1 = px.line(df_data, x="Date", y="Obs")
fig1.show()
fig2 = px.line(df_data, x="Date", y="precipwsnow")
fig2.show()
fig3 = px.line(df_data, x="Date", y="PET")
fig3.show()

In [5]:
precip = df_data["precipwsnow"].values
pet = df_data["PET"].values
gwl = df_data["Obs"].values

In [6]:
# Concatenate the features
features_arr = np.column_stack((precip, pet))
features_arr.shape

(9400, 2)

In [7]:
# Normalize the features
scaler = MinMaxScaler(feature_range=(-1, 1))
features_scaled_arr = scaler.fit_transform(features_arr)
features_scaled_arr.shape

(9400, 2)

In [8]:
# Normalise the target
target_scaler = MinMaxScaler(feature_range=(-1, 1))
gwl_scaled_arr = target_scaler.fit_transform(gwl.reshape(-1, 1))

In [9]:
def create_sequences(data, seq_length):
    """
    Transforms time-series data into sequences of a specified length.

    Parameters:
    data (np.array): A 2D numpy array where each row is a time step and each column is a feature.
    seq_length (int): The number of time steps to include in each output sequence.

    Returns:
    np.array: A 3D numpy array of shape (num_samples - seq_length + 1, seq_length, num_features).
    """

    xs = []  # Initialise an empty list to store sequences

    # For each possible sequence in the data...
    for i in range(len(data) - seq_length + 1):
        # Extract a sequence of length `seq_length`
        x = data[i:(i+seq_length)]
        # Append the sequence to the list
        xs.append(x)

    # Convert the list of sequences into a 3D numpy array
    return np.array(xs)

seq_length = 200
features_seq_arr = create_sequences(features_scaled_arr, seq_length)
# Also need to make sure the first 365 elements of the gwl array are clipped
gwl_arr = gwl_scaled_arr[seq_length - 1:]
print(features_seq_arr.shape)
print(gwl_arr.shape)

(9201, 200, 2)
(9201, 1)


In [10]:
features_tensor = torch.from_numpy(features_seq_arr).float()
gwl_tensor = torch.from_numpy(gwl_arr).float()#.unsqueeze(1)
print(features_tensor.shape)
print(gwl_tensor.shape)

torch.Size([9201, 200, 2])
torch.Size([9201, 1])


In [11]:
# Split into training and test sets
train_size = int(len(features_tensor) * 0.8)
test_size = len(features_tensor) - train_size

features_train = features_tensor[:train_size]
features_test = features_tensor[train_size:]
gwl_train  = gwl_tensor[:train_size]
gwl_test = gwl_tensor[train_size:]

print(f"{features_train.shape}: {gwl_train.shape}")
print(f"{features_test.shape}: {gwl_test.shape}")

torch.Size([7360, 200, 2]): torch.Size([7360, 1])
torch.Size([1841, 200, 2]): torch.Size([1841, 1])


In [12]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

# Work on this tomorrow
train_dataset = TimeSeriesDataset(features_train, gwl_train)
test_dataset = TimeSeriesDataset(features_test, gwl_test)

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        # self.lstm.reset_parameters()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [14]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
# Initialize the model, loss function, and optimizer
model = LSTM(input_size=2, hidden_size=20, num_layers=2, output_size=1).to(device)

class NSELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, predictions: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        denominator = torch.sum((targets - torch.mean(targets)) ** 2)
        numerator = torch.sum((targets - predictions) ** 2)
        nse_val = numerator / denominator
        return nse_val

criterion = nn.MSELoss()
# criterion = NSELoss()

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [17]:
# Training function
def train_epoch(epoch: int):
    model.train()
    running_loss = 0
    for batch in train_loader:
        # batch is a list with two elements
        # The first element is the feature array
        # The second element is the GWL array
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        output = model(x_batch)
        loss = criterion(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss_across_batches = running_loss / len(test_loader)

    # print(f'Epoch: {epoch + 1} Final Batch, Loss: {running_loss}')
    return avg_loss_across_batches


In [18]:
# Testing
def test_epoch(epoch: int):
    model.eval()
    running_loss = 0
    for i, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        with torch.no_grad():
            output = model(x_batch)
            loss = criterion(output, y_batch)
            running_loss += loss.item()
    avg_loss_across_batches = running_loss / len(test_loader)
    
    # print(f'Epoch: {epoch + 1} Test Loss: {avg_loss_across_batches}')
    # print('***************************************************')
    print()
    return avg_loss_across_batches

In [19]:
def nse(predictions: np.ndarray, targets: np.ndarray) -> float:
    denominator = np.sum((targets - np.mean(targets)) ** 2)
    numerator = np.sum((targets - predictions) ** 2)
    nse_val = 1 - numerator / denominator
    return nse_val

In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
num_epochs = 1000
train_loss = []
test_loss = []
for epoch in range(num_epochs):
    train_loss.append(train_epoch(epoch))
    test_loss.append(test_epoch(epoch))
    with torch.no_grad():
        train_predicted = model(features_train.to(device)).to('cpu').numpy()
        test_predicted = model(features_test.to(device)).to('cpu').numpy()
    
    gwl_train_inverse = target_scaler.inverse_transform(gwl_train)
    gwl_test_inverse = target_scaler.inverse_transform(gwl_test)
    predicted_train_inverse = target_scaler.inverse_transform(train_predicted)
    predicted_test_inverse = target_scaler.inverse_transform(test_predicted)

    train_nse = nse(predicted_train_inverse.flatten(), gwl_train_inverse.flatten())
    test_nse = nse(predicted_test_inverse.flatten(), gwl_test_inverse.flatten())
    print(f"Epoch {epoch + 1} Train NSE: {train_nse}")
    print(f"Epoch {epoch + 1}  Test NSE: {test_nse}")
    

    if (epoch + 1) != num_epochs:
        continue
    df = pd.DataFrame()
    df["Training Observed"] = np.append(gwl_train_inverse.flatten(), np.full(len(gwl_test), np.nan))
    df["Training Predicted"] = np.append(predicted_train_inverse.flatten(), np.full(len(gwl_test), np.nan))
    df["Testing Predicted"] = np.append(np.full(len(gwl_train), np.nan), predicted_test_inverse)
    df["Testing Observed"] = np.append(np.full(len(gwl_train), np.nan), gwl_test_inverse)
    fig = px.line(df)
    fig.show()

    fig = px.line(np.array(train_loss))
    fig.show()
    fig = px.line(np.array(test_loss))
    fig.show()



Epoch 1 Train NSE: 0.8026824261392275
Epoch 1  Test NSE: 0.8398833595121531

Epoch 2 Train NSE: 0.8027813848578953
Epoch 2  Test NSE: 0.8400199403077568

Epoch 3 Train NSE: 0.8027782851904626
Epoch 3  Test NSE: 0.8400346074065306

Epoch 4 Train NSE: 0.8026865312956657
Epoch 4  Test NSE: 0.8397677821391918

Epoch 5 Train NSE: 0.8026261130561216
Epoch 5  Test NSE: 0.839556810861612

Epoch 6 Train NSE: 0.8026427666195441
Epoch 6  Test NSE: 0.8394920713024564

Epoch 7 Train NSE: 0.8026586282429033
Epoch 7  Test NSE: 0.8395115290336433

Epoch 8 Train NSE: 0.802712184681923
Epoch 8  Test NSE: 0.8396244269263002

Epoch 9 Train NSE: 0.8027430631745704
Epoch 9  Test NSE: 0.8397438518130839

Epoch 10 Train NSE: 0.8027730257723822
Epoch 10  Test NSE: 0.8398192228943833

Epoch 11 Train NSE: 0.8027567434259746
Epoch 11  Test NSE: 0.8397935255621619

Epoch 12 Train NSE: 0.802744791483217
Epoch 12  Test NSE: 0.8397259774909146

Epoch 13 Train NSE: 0.8027131086284716
Epoch 13  Test NSE: 0.83963142901

In [33]:
torch.save(model.state_dict(), 'single_catchment_model.pt')
# model.load_state_dict(torch.load('single_catchment_model.pt'))

In [29]:
def count_parameters(model: LSTM) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model)} parameters")

The model has 5301 parameters
